In [2]:
!pip install --upgrade jaxlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached jaxlib-0.3.10-cp310-none-manylinux2014_x86_64.whl (69.0 MB)
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.75
    Uninstalling jaxlib-0.1.75:
      Successfully uninstalled jaxlib-0.1.75
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
surrojax-gp 0.0.0 requires jax==0.2.25, but you have jax 0.3.13 which is incompatible.
surrojax-gp 0.0.0 requires jaxlib==0.1.75, but you have jaxlib 0.3.10 which is incompatible.


In [4]:
import jax
import jax.numpy as jnp
import numpy as np

In [5]:
from arraytainers import Arraytainer, Jaxtainer

In [6]:
a = Arraytainer({'a': (1,1), 'b':(2,1)})

In [9]:
np.concatenate((np.array([1]), a))

Arraytainer({'a': array([1, 1, 1]), 'b': array([1, 2, 1])})

In [2]:
value_structured = {'a': jnp.array(1.), 0:(2., 3.)}
value_flat, value_tree = jax.tree_flatten(value_structured)

NameError: name 'jnp' is not defined

In [15]:
jax.tree_flatten({'a': jnp.array([3., 4.]), '0': jnp.array([[1., 2.],[3., 4.]]) })

([DeviceArray([[1., 2.],
               [3., 4.]], dtype=float32),
  DeviceArray([3., 4.], dtype=float32)],
 PyTreeDef({'0': *, 'a': *}))

In [4]:
func = lambda x: x[0]**2 + np.log(x[0])
vmapped_func = jax.vmap(func, in_axes=0)
jaxtainer = Jaxtainer({'a': jnp.array([3, 4]), 0: jnp.array([[1, 2],[3, 4]])})
output = vmapped_func(jaxtainer)

TypeError: '<' not supported between instances of 'int' and 'str'

In [5]:
val = Arraytainer({'a': 'C', 'b': [{'a': 'F'}]}, array_conversions=False)

In [6]:
val.flatten()

array(['C', 'F'], dtype='<U1')

In [6]:
val._recursive_arraytainer_conversion(val, dtype=None, array_conversions=False)

Arraytainer({'a': 'C', 'b': [{'a': array('F', dtype='<U1')}]})
False
a


Arraytainer({'a': 'C', 'b': [{'a': array('F', dtype='<U1')}]})